# Library import

In [1]:
""" os """
import os

""" torch """
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torch.utils.data.sampler import SubsetRandomSampler


"""tensor board"""
import torchvision
#from torch.utils.tensorboard import SummaryWriter


"""glob"""
from glob import glob

""" tqdm """
import time
from tqdm import tqdm

"""Pandas"""
import pandas as pd

""" numpy """
import numpy as np
from numpy import argmax
from PIL import Image

"""JSON"""
import json

"""sklearn"""
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, r2_score

"""seaborn"""
import seaborn as sns

"""scipy"""
from scipy import io
from scipy import signal
from scipy.fft import fft, ifft,fftfreq
from scipy import stats

"""SUMMARY"""
from torchsummary import summary

"""time"""
import time


"""pingouin"""
import pingouin as pg

import re
import shutil
import random
import matplotlib.pyplot as plt
import scipy

# Path init

In [ ]:
os.listdir()

In [4]:
cur_path = "D:\Dataset [ECG DB data 500Hz]"
os.chdir(cur_path)
os.listdir()

['demographics(bmi)_label_500Hz.json',
 'demographics(bmi)_label_500Hz_1Lead.json',
 'demographics(gender,age,ABO)_label_500Hz.json',
 'demographics(gender,age,ABO)_label_500Hz_1Lead.json',
 'ECG Dataset_250Hz',
 'ECG Dataset_500Hz',
 'ECG Dataset_500Hz_1Lead',
 'ECG Dataset_500Hz_original',
 'ECG Dataset_MissingLead',
 'EKG_decrypt_1032442rec_296989pt',
 'EKG_decrypt_1032442rec_296989pt.zip',
 'signal.jpg']

In [5]:
experiment_num = 
fig_save_path = 'path'

In [6]:
folder_path = fig_save_path + str(experiment_num)

try:
    if not(os.path.isdir(folder_path)):
        os.makedirs(os.path.join(folder_path))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise

# Hyper parameters

In [49]:
seed = 1

#validation ratio
validation_ratio = 0.1

#learning rate
lr = 0.001
 

momentum = 0.5


batch_size = 512    
test_batch_size = 512


epochs = 50
no_cuda = False

log_interval = 5

# Set the seed and GPU

In [50]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers':0,'pin_memory':True} if use_cuda else {}

# Model

In [52]:
class ECG_CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(ECG_CNN, self).__init__()

        """Convolution"""
        self.conv1 = nn.Conv1d(12, 48, kernel_size=5, stride=1 )
        self.conv2 = nn.Conv1d(48, 96, kernel_size=5, stride=1 )
        self.conv3 = nn.Conv1d(96, 192, kernel_size=5, stride=1 )
        
        """BatchNormalize"""
        self.bn1 = nn.BatchNorm1d(48)
        self.bn2 = nn.BatchNorm1d(96)
        self.bn3 = nn.BatchNorm1d(192)
        
        """ReLU"""
        self.relu = nn.ReLU(inplace=True)
        
        """Sigmoid"""
        self.sigmoid = nn.Sigmoid()
        
        """ MaxPooling"""     
        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        """ Global Max, Average Pooling"""
        #self.globalmaxpool1 = nn.MaxPool1d(246, stride=1)
        self.globalavrpool1 = nn.AvgPool1d(kernel_size = 621, stride=1)
        
        """Drop out"""
        self.dp = nn.Dropout(p=0.5)
        
        
        """Fully Connected """ 
        self.fc1 = nn.Linear(1*192, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes) 
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        #x = self.globalmaxpool1(x)
        x = self.globalavrpool1(x)
        
        
        """Fully Connected"""
        x = x.view(-1, 1*192*1) .
        
        
        # Dense
        x = F.relu(self.fc1(x))
        #x = self.dp(x)
        x = F.relu(self.fc2(x))
        #x = self.dp(x)
        x = self.fc3(x)
        
        #x = self.sigmoid(x)
        
        return x

# Data Path(12 Lead)

In [53]:
with open('D:\Dataset [ECG DB data 500Hz]/demographics(gender,age,ABO)_label_500Hz.json', 'r') as f:
    label_data = json.load(f)

In [54]:
len(label_data)

124415

In [55]:
label_data[0]

{'data_path': 'ECG Dataset_500Hz\\pt_no = 0006505, date = 12-17-2013, srate = 500.npy',
 'pt_no': '0006505',
 'gender': '2',
 'age': 76,
 'ABO': 'AB'}

In [56]:
data_paths_infant,data_paths_10s,data_paths_20s,data_paths_30s,data_paths_40s,data_paths_50s,data_paths_60s  = [],[],[],[],[],[],[]
data_paths_70s, data_paths_80s, data_paths_90s = [],[],[]

for i in range(0,len(label_data),1):
    
    """get label"""
    tempt = label_data[i]['age']
    
    if tempt<18:
        data_paths_infant.append(i)
    elif 18 <= tempt < 20:
        data_paths_10s.append(i)
    elif 20 <= tempt < 30:
        data_paths_20s.append(i)
    elif 30 <= tempt < 40:
        data_paths_30s.append(i)
    elif 40 <= tempt < 50:
        data_paths_40s.append(i)
    elif 50 <= tempt < 60:
        data_paths_50s.append(i)
    elif 60 <= tempt < 70:
        data_paths_60s.append(i)
    elif 70 <= tempt < 80:
        data_paths_70s.append(i)
    elif 80 <= tempt < 90:
        data_paths_80s.append(i)
    else:
        data_paths_90s.append(i)

# train, validatin, test set 분류

In [57]:
def Make_train_validation_test_paths(data_paths, train_ratio, valid_ratio):
    
    """count number of data paths"""
    num_datapaths = len(data_paths)
    indices = list(range(num_datapaths))
    
    train_split = int(np.floor(train_ratio * num_datapaths))
    valid_split = int(np.floor((train_ratio+valid_ratio) * num_datapaths))

    """set the seed and shuffle"""
    np.random.seed(seed)
    np.random.shuffle(indices)
    
    """spllit data paths"""
    train_data_paths      = []
    validation_data_paths = []
    test_data_pahts       = []
    
    
    train_idx, valid_idx, test_idx = indices[:train_split], indices[train_split:valid_split], indices[valid_split:]
    

    for i in range(0,len(train_idx),1):
        train_data_paths.append( data_paths[train_idx[i]])
        
    for i in range(0,len(valid_idx),1):
        validation_data_paths.append( data_paths[valid_idx[i]])
        
    for i in range(0,len(test_idx),1):
        test_data_pahts.append( data_paths[test_idx[i]])
        
    
    
    return train_data_paths, validation_data_paths, test_data_pahts

In [58]:
train_paths_10s, validation_paths_10s, test_paths_10s = Make_train_validation_test_paths(data_paths_10s,0.4,0.3)
train_paths_20s, validation_paths_20s, test_paths_20s = Make_train_validation_test_paths(data_paths_20s,0.4,0.3)
train_paths_30s, validation_paths_30s, test_paths_30s = Make_train_validation_test_paths(data_paths_30s,0.4,0.3)
train_paths_40s, validation_paths_40s, test_paths_40s = Make_train_validation_test_paths(data_paths_40s,0.4,0.3)
train_paths_50s, validation_paths_50s, test_paths_50s = Make_train_validation_test_paths(data_paths_50s,0.4,0.3)
train_paths_60s, validation_paths_60s, test_paths_60s = Make_train_validation_test_paths(data_paths_60s,0.4,0.3)
train_paths_70s, validation_paths_70s, test_paths_70s = Make_train_validation_test_paths(data_paths_70s,0.4,0.3)
train_paths_80s, validation_paths_80s, test_paths_80s = Make_train_validation_test_paths(data_paths_80s,0.4,0.3)
train_paths_90s, validation_paths_90s, test_paths_90s = Make_train_validation_test_paths(data_paths_90s,0.4,0.3)

In [59]:
train_paths      = train_paths_10s + train_paths_20s + train_paths_30s + train_paths_40s + train_paths_50s + train_paths_60s + train_paths_70s + train_paths_80s + train_paths_90s
validation_paths = validation_paths_10s + validation_paths_20s + validation_paths_30s + validation_paths_40s + validation_paths_50s + validation_paths_60s + validation_paths_70s + validation_paths_80s + validation_paths_90s
test_paths       = test_paths_10s + test_paths_20s + test_paths_30s + test_paths_40s + test_paths_50s + test_paths_60s + test_paths_70s + test_paths_80s + test_paths_90s
len(train_paths), len(validation_paths), len(test_paths)

(49762, 37324, 37329)

# Custom Dataset

In [60]:
class CustomDataset(Dataset):

    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform

    def __getitem__(self, idx):
          
        """get data"""
        tempt = self.data_paths[idx] 
        path  = label_data[tempt]['data_path']
        data = np.load(file = path)
        
        
        """get label"""
        
        if label_data[tempt]['age'] <= 40:
            label = 0
        else:
            label = 1
        

        return data,label
    
    
    def __len__(self):
        return len(self.data_paths)

In [61]:
train_dataset      = CustomDataset(train_paths,transforms.Compose([transforms.ToTensor()]))
validation_dataset = CustomDataset(validation_paths,transforms.Compose([transforms.ToTensor()]))
test_dataset       = CustomDataset(test_paths,transforms.Compose([transforms.ToTensor()]))

print(len(train_dataset),len(validation_dataset), len(test_dataset))

49762 37324 37329


# Data Loader

In [62]:
"""Train"""
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size = batch_size,
    #sampler = train_sampler,
    shuffle = True, 
    **kwargs
)

"""Validation"""
validation_loader = torch.utils.data.DataLoader(
    dataset=validation_dataset,
    batch_size = batch_size,
    #sampler = valid_sampler,
    shuffle = True,
    **kwargs
)

"""Test"""
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size = test_batch_size,
    shuffle = True,
    **kwargs
)


In [63]:
print("Length of the train_loader:", len(train_loader))
print("Length of the val_loader:", len(validation_loader))
print("Length of the test_loader:", len(test_loader))

Length of the train_loader: 98
Length of the val_loader: 73
Length of the test_loader: 73


# Optimizer

In [64]:
model = ECG_CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=lr)

# Train

## init

In [23]:
"""Train"""
train_losses        = []
avg_train_losses    = []
Train_baths_ACC     = [] 
Train_ACC           = [] 


"""Validaion"""
valid_losses        = []
avg_valid_losses    = []
Validation_ACC      = []
Valid_ACC_per_Class = []


In [65]:
num_classes = 2
criterion = nn.CrossEntropyLoss()

In [25]:
"""save best model"""
best_acc = 0
best_model_save_path = 'path'
best_model_save_path = best_model_save_path + str(experiment_num)

In [26]:
for epoch in range(1, epochs + 1):
    
    """Train"""
    model.train()
    
    train_loss = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        
        data, target = data.to(device, dtype=torch.float), target.to(device, dtype=torch.long)
        optimizer.zero_grad()
        
        output = model(data)
        
        #print(output)

        """pred(Cross Entropy)"""
        #pred = output.argmax(dim=1, keepdim=True)  #cross entropy
        pred = F.softmax(output,dim =1).argmax(dim=1, keepdim=True)
        
        """loss"""
        loss = criterion(output, target)
        train_loss += loss.item()

        loss.backward() # 가중치 갱신.
        optimizer.step()
        
        
        correct = 0
        total = target.size(0)
        correct += pred.eq(target.view_as(pred)).sum().item()
        accuracy = 100. * correct / total
        Train_baths_ACC.append(accuracy)
        
        
        if batch_idx % log_interval == 0:
            #1.
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

            #2.
            print('Train set:  batch loss: {:.4f}, Accuracy: {:.0f}% '.format(
                loss.item() ,accuracy))

    
    """Validation"""
    model.eval()
    
    valid_loss = 0
    correct = 0
    total = len(validation_loader.dataset)


    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(validation_loader):
            data, target = data.to(device, dtype=torch.float), target.to(device, dtype=torch.long)
            
            output = model(data)


            """pred(Cross Entropy)"""
            #pred = output.argmax(dim=1, keepdim=True) # cross entropy
            pred = F.softmax(output,dim =1).argmax(dim=1, keepdim=True) # 확률로 변환 후 argmax 적용.
            
            """loss"""
            loss = criterion(output, target)
            valid_loss += loss.item()
        
            correct += pred.eq(target.view_as(pred)).sum().item()
            

                
    """Loss and ACC """
    
    train_loss /= len(train_loader)
    valid_loss /= len(validation_loader)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    
    Train_ACC.append(sum(Train_baths_ACC)/len(Train_baths_ACC))
    Train_baths_ACC = []
    
    valid_accuracy = 100. * correct / total
    Validation_ACC.append(valid_accuracy)

    
    print('------------------------------------------')
    print('Valid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        valid_loss, correct, total, valid_accuracy))
    print('-------------------------------------------')
    
    
    """Save best model"""
    
    if valid_accuracy > best_acc:
        torch.save(model, best_model_save_path)
        print("model saved.")
        print('-------------------------------------------')
        best_acc = valid_accuracy
    
    

Train Epoch: 1 [0/49762 (0%)]	Loss: 0.641442
Train set:  batch loss: 0.6414, Accuracy: 75% 
Train Epoch: 1 [2560/49762 (5%)]	Loss: 0.516673
Train set:  batch loss: 0.5167, Accuracy: 79% 
Train Epoch: 1 [5120/49762 (10%)]	Loss: 0.494752
Train set:  batch loss: 0.4948, Accuracy: 78% 
Train Epoch: 1 [7680/49762 (15%)]	Loss: 0.451035
Train set:  batch loss: 0.4510, Accuracy: 79% 
Train Epoch: 1 [10240/49762 (20%)]	Loss: 0.383461
Train set:  batch loss: 0.3835, Accuracy: 83% 
Train Epoch: 1 [12800/49762 (26%)]	Loss: 0.430272
Train set:  batch loss: 0.4303, Accuracy: 79% 
Train Epoch: 1 [15360/49762 (31%)]	Loss: 0.428526
Train set:  batch loss: 0.4285, Accuracy: 78% 
Train Epoch: 1 [17920/49762 (36%)]	Loss: 0.403112
Train set:  batch loss: 0.4031, Accuracy: 78% 
Train Epoch: 1 [20480/49762 (41%)]	Loss: 0.407507
Train set:  batch loss: 0.4075, Accuracy: 79% 
Train Epoch: 1 [23040/49762 (46%)]	Loss: 0.336712
Train set:  batch loss: 0.3367, Accuracy: 86% 
Train Epoch: 1 [25600/49762 (51%)]	Loss

Train Epoch: 4 [48640/49762 (97%)]	Loss: 0.367751
Train set:  batch loss: 0.3678, Accuracy: 83% 
------------------------------------------
Valid set: Average loss: 0.3169, Accuracy: 32009/37324 (86%)
-------------------------------------------
모델이 저장되었습니다.
-------------------------------------------
Train Epoch: 5 [0/49762 (0%)]	Loss: 0.326239
Train set:  batch loss: 0.3262, Accuracy: 86% 
Train Epoch: 5 [2560/49762 (5%)]	Loss: 0.294523
Train set:  batch loss: 0.2945, Accuracy: 86% 
Train Epoch: 5 [5120/49762 (10%)]	Loss: 0.323364
Train set:  batch loss: 0.3234, Accuracy: 86% 
Train Epoch: 5 [7680/49762 (15%)]	Loss: 0.323352
Train set:  batch loss: 0.3234, Accuracy: 85% 
Train Epoch: 5 [10240/49762 (20%)]	Loss: 0.287757
Train set:  batch loss: 0.2878, Accuracy: 89% 
Train Epoch: 5 [12800/49762 (26%)]	Loss: 0.338699
Train set:  batch loss: 0.3387, Accuracy: 85% 
Train Epoch: 5 [15360/49762 (31%)]	Loss: 0.337974
Train set:  batch loss: 0.3380, Accuracy: 85% 
Train Epoch: 5 [17920/49762 

Train Epoch: 8 [43520/49762 (87%)]	Loss: 0.293494
Train set:  batch loss: 0.2935, Accuracy: 88% 
Train Epoch: 8 [46080/49762 (92%)]	Loss: 0.329483
Train set:  batch loss: 0.3295, Accuracy: 85% 
Train Epoch: 8 [48640/49762 (97%)]	Loss: 0.307099
Train set:  batch loss: 0.3071, Accuracy: 86% 
------------------------------------------
Valid set: Average loss: 0.3144, Accuracy: 32097/37324 (86%)
-------------------------------------------
Train Epoch: 9 [0/49762 (0%)]	Loss: 0.325000
Train set:  batch loss: 0.3250, Accuracy: 85% 
Train Epoch: 9 [2560/49762 (5%)]	Loss: 0.295736
Train set:  batch loss: 0.2957, Accuracy: 87% 
Train Epoch: 9 [5120/49762 (10%)]	Loss: 0.296076
Train set:  batch loss: 0.2961, Accuracy: 87% 
Train Epoch: 9 [7680/49762 (15%)]	Loss: 0.308658
Train set:  batch loss: 0.3087, Accuracy: 85% 
Train Epoch: 9 [10240/49762 (20%)]	Loss: 0.360391
Train set:  batch loss: 0.3604, Accuracy: 85% 
Train Epoch: 9 [12800/49762 (26%)]	Loss: 0.311187
Train set:  batch loss: 0.3112, Acc

Train Epoch: 12 [38400/49762 (77%)]	Loss: 0.291286
Train set:  batch loss: 0.2913, Accuracy: 86% 
Train Epoch: 12 [40960/49762 (82%)]	Loss: 0.310298
Train set:  batch loss: 0.3103, Accuracy: 86% 
Train Epoch: 12 [43520/49762 (87%)]	Loss: 0.307146
Train set:  batch loss: 0.3071, Accuracy: 87% 
Train Epoch: 12 [46080/49762 (92%)]	Loss: 0.287182
Train set:  batch loss: 0.2872, Accuracy: 88% 
Train Epoch: 12 [48640/49762 (97%)]	Loss: 0.334359
Train set:  batch loss: 0.3344, Accuracy: 85% 
------------------------------------------
Valid set: Average loss: 0.2997, Accuracy: 32308/37324 (87%)
-------------------------------------------
모델이 저장되었습니다.
-------------------------------------------
Train Epoch: 13 [0/49762 (0%)]	Loss: 0.341594
Train set:  batch loss: 0.3416, Accuracy: 87% 
Train Epoch: 13 [2560/49762 (5%)]	Loss: 0.281677
Train set:  batch loss: 0.2817, Accuracy: 87% 
Train Epoch: 13 [5120/49762 (10%)]	Loss: 0.290453
Train set:  batch loss: 0.2905, Accuracy: 89% 
Train Epoch: 13 [76

Train Epoch: 16 [30720/49762 (61%)]	Loss: 0.311501
Train set:  batch loss: 0.3115, Accuracy: 86% 
Train Epoch: 16 [33280/49762 (66%)]	Loss: 0.344237
Train set:  batch loss: 0.3442, Accuracy: 84% 
Train Epoch: 16 [35840/49762 (71%)]	Loss: 0.292965
Train set:  batch loss: 0.2930, Accuracy: 87% 
Train Epoch: 16 [38400/49762 (77%)]	Loss: 0.299626
Train set:  batch loss: 0.2996, Accuracy: 87% 
Train Epoch: 16 [40960/49762 (82%)]	Loss: 0.309541
Train set:  batch loss: 0.3095, Accuracy: 85% 
Train Epoch: 16 [43520/49762 (87%)]	Loss: 0.257320
Train set:  batch loss: 0.2573, Accuracy: 89% 
Train Epoch: 16 [46080/49762 (92%)]	Loss: 0.333293
Train set:  batch loss: 0.3333, Accuracy: 86% 
Train Epoch: 16 [48640/49762 (97%)]	Loss: 0.307008
Train set:  batch loss: 0.3070, Accuracy: 85% 
------------------------------------------
Valid set: Average loss: 0.3122, Accuracy: 32148/37324 (86%)
-------------------------------------------
Train Epoch: 17 [0/49762 (0%)]	Loss: 0.260177
Train set:  batch loss

...

Train Epoch: 32 [7680/49762 (15%)]	Loss: 0.276922
Train set:  batch loss: 0.2769, Accuracy: 88% 
Train Epoch: 32 [10240/49762 (20%)]	Loss: 0.287344
Train set:  batch loss: 0.2873, Accuracy: 86% 
Train Epoch: 32 [12800/49762 (26%)]	Loss: 0.252502
Train set:  batch loss: 0.2525, Accuracy: 90% 
Train Epoch: 32 [15360/49762 (31%)]	Loss: 0.290200
Train set:  batch loss: 0.2902, Accuracy: 87% 
Train Epoch: 32 [17920/49762 (36%)]	Loss: 0.201246
Train set:  batch loss: 0.2012, Accuracy: 92% 
Train Epoch: 32 [20480/49762 (41%)]	Loss: 0.245069
Train set:  batch loss: 0.2451, Accuracy: 89% 
Train Epoch: 32 [23040/49762 (46%)]	Loss: 0.247134
Train set:  batch loss: 0.2471, Accuracy: 89% 
Train Epoch: 32 [25600/49762 (51%)]	Loss: 0.280175
Train set:  batch loss: 0.2802, Accuracy: 88% 
Train Epoch: 32 [28160/49762 (56%)]	Loss: 0.295184
Train set:  batch loss: 0.2952, Accuracy: 87% 
Train Epoch: 32 [30720/49762 (61%)]	Loss: 0.239268
Train set:  batch loss: 0.2393, Accuracy: 89% 
Train Epoch: 32 [3328

Train Epoch: 36 [2560/49762 (5%)]	Loss: 0.276606
Train set:  batch loss: 0.2766, Accuracy: 87% 
Train Epoch: 36 [5120/49762 (10%)]	Loss: 0.313134
Train set:  batch loss: 0.3131, Accuracy: 86% 
Train Epoch: 36 [7680/49762 (15%)]	Loss: 0.286028
Train set:  batch loss: 0.2860, Accuracy: 87% 
Train Epoch: 36 [10240/49762 (20%)]	Loss: 0.261663
Train set:  batch loss: 0.2617, Accuracy: 88% 
Train Epoch: 36 [12800/49762 (26%)]	Loss: 0.241398
Train set:  batch loss: 0.2414, Accuracy: 89% 
Train Epoch: 36 [15360/49762 (31%)]	Loss: 0.545820
Train set:  batch loss: 0.5458, Accuracy: 70% 
Train Epoch: 36 [17920/49762 (36%)]	Loss: 0.285582
Train set:  batch loss: 0.2856, Accuracy: 86% 
Train Epoch: 36 [20480/49762 (41%)]	Loss: 0.323406
Train set:  batch loss: 0.3234, Accuracy: 87% 
Train Epoch: 36 [23040/49762 (46%)]	Loss: 0.321435
Train set:  batch loss: 0.3214, Accuracy: 84% 
Train Epoch: 36 [25600/49762 (51%)]	Loss: 0.310839
Train set:  batch loss: 0.3108, Accuracy: 86% 
Train Epoch: 36 [28160/4

Train Epoch: 40 [0/49762 (0%)]	Loss: 0.250940
Train set:  batch loss: 0.2509, Accuracy: 89% 
Train Epoch: 40 [2560/49762 (5%)]	Loss: 0.304598
Train set:  batch loss: 0.3046, Accuracy: 86% 
Train Epoch: 40 [5120/49762 (10%)]	Loss: 0.225867
Train set:  batch loss: 0.2259, Accuracy: 90% 
Train Epoch: 40 [7680/49762 (15%)]	Loss: 0.262860
Train set:  batch loss: 0.2629, Accuracy: 89% 
Train Epoch: 40 [10240/49762 (20%)]	Loss: 0.237853
Train set:  batch loss: 0.2379, Accuracy: 89% 
Train Epoch: 40 [12800/49762 (26%)]	Loss: 0.233817
Train set:  batch loss: 0.2338, Accuracy: 90% 
Train Epoch: 40 [15360/49762 (31%)]	Loss: 0.255140
Train set:  batch loss: 0.2551, Accuracy: 89% 
Train Epoch: 40 [17920/49762 (36%)]	Loss: 0.264886
Train set:  batch loss: 0.2649, Accuracy: 88% 
Train Epoch: 40 [20480/49762 (41%)]	Loss: 0.244365
Train set:  batch loss: 0.2444, Accuracy: 89% 
Train Epoch: 40 [23040/49762 (46%)]	Loss: 0.220658
Train set:  batch loss: 0.2207, Accuracy: 91% 
Train Epoch: 40 [25600/49762 

------------------------------------------
Valid set: Average loss: 0.2841, Accuracy: 32634/37324 (87%)
-------------------------------------------
Train Epoch: 44 [0/49762 (0%)]	Loss: 0.230553
Train set:  batch loss: 0.2306, Accuracy: 91% 
Train Epoch: 44 [2560/49762 (5%)]	Loss: 0.300809
Train set:  batch loss: 0.3008, Accuracy: 85% 
Train Epoch: 44 [5120/49762 (10%)]	Loss: 0.231817
Train set:  batch loss: 0.2318, Accuracy: 90% 
Train Epoch: 44 [7680/49762 (15%)]	Loss: 0.256317
Train set:  batch loss: 0.2563, Accuracy: 90% 
Train Epoch: 44 [10240/49762 (20%)]	Loss: 0.260343
Train set:  batch loss: 0.2603, Accuracy: 90% 
Train Epoch: 44 [12800/49762 (26%)]	Loss: 0.247059
Train set:  batch loss: 0.2471, Accuracy: 89% 
Train Epoch: 44 [15360/49762 (31%)]	Loss: 0.247732
Train set:  batch loss: 0.2477, Accuracy: 89% 
Train Epoch: 44 [17920/49762 (36%)]	Loss: 0.264600
Train set:  batch loss: 0.2646, Accuracy: 87% 
Train Epoch: 44 [20480/49762 (41%)]	Loss: 0.285220
Train set:  batch loss: 0.

Train Epoch: 47 [46080/49762 (92%)]	Loss: 0.221706
Train set:  batch loss: 0.2217, Accuracy: 91% 
Train Epoch: 47 [48640/49762 (97%)]	Loss: 0.265323
Train set:  batch loss: 0.2653, Accuracy: 87% 
------------------------------------------
Valid set: Average loss: 0.2807, Accuracy: 32743/37324 (88%)
-------------------------------------------
모델이 저장되었습니다.
-------------------------------------------
Train Epoch: 48 [0/49762 (0%)]	Loss: 0.226408
Train set:  batch loss: 0.2264, Accuracy: 90% 
Train Epoch: 48 [2560/49762 (5%)]	Loss: 0.341813
Train set:  batch loss: 0.3418, Accuracy: 84% 
Train Epoch: 48 [5120/49762 (10%)]	Loss: 0.225229
Train set:  batch loss: 0.2252, Accuracy: 91% 
Train Epoch: 48 [7680/49762 (15%)]	Loss: 0.257705
Train set:  batch loss: 0.2577, Accuracy: 88% 
Train Epoch: 48 [10240/49762 (20%)]	Loss: 0.268009
Train set:  batch loss: 0.2680, Accuracy: 88% 
Train Epoch: 48 [12800/49762 (26%)]	Loss: 0.239924
Train set:  batch loss: 0.2399, Accuracy: 91% 
Train Epoch: 48 [153

# Model save and Load

In [44]:
save_path = 'path'

In [45]:
save_path = save_path + str(experiment_num)

In [46]:
torch.save(model, save_path)